# Test to see if VAE can produce a good model

In [195]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import gym

In [196]:
from vae_recurrent import VAE, create_decoder, create_encoder
from util import random_observation_sequence, transform_observations

In [200]:
env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

observation_max = np.array([0.6, 0.07])
observation_min = np.array([-1.2, -0.07])

all_observations = []
all_observations_scaled = []
num_episodes = 10

obs_stddev = [0.05, 0.05]
# obs_stddev = [0, 0]

for i in range(num_episodes):
    o, a, r = random_observation_sequence(env, 1000, epsilon=0.1)
    # o_scaled = transform_observations(o, observation_max, observation_min, [0.05, 0.05])
    o_scaled = transform_observations(o, observation_max, observation_min, obs_stddev)
    o_scaled

    o = transform_observations(o, observation_max, observation_min, [0, 0])

    all_observations_scaled.append(o_scaled)
    all_observations.append(o)

all_observations = np.vstack(all_observations)
all_observations_scaled = np.vstack(all_observations_scaled)
all_observations.shape

(4834, 2)

In [201]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])

vae = VAE(enc, dec, [0, 0], [0.3, 0.3], llik_scaling=1, kl_scaling=1)
vae.compile(optimizer=tf.keras.optimizers.Adam())

In [202]:
vae.fit(all_observations_scaled, epochs=5, batch_size=12)

Epoch 1/5


2022-08-07 14:12:29.063746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


403/403 [==============================] - 5s 9ms/step - loss: 0.6870 - reconstruction_loss: 0.6785 - kl_loss: 5.4863e-04
Epoch 2/5
403/403 [==============================] - 4s 9ms/step - loss: 0.6294 - reconstruction_loss: 0.6108 - kl_loss: 8.6298e-04
Epoch 3/5
403/403 [==============================] - 4s 9ms/step - loss: 0.5930 - reconstruction_loss: 0.5910 - kl_loss: 0.0012
Epoch 4/5
403/403 [==============================] - 4s 9ms/step - loss: 0.5904 - reconstruction_loss: 0.5904 - kl_loss: 0.0012
Epoch 5/5
403/403 [==============================] - 4s 9ms/step - loss: 0.5900 - reconstruction_loss: 0.5903 - kl_loss: 0.0012


In [185]:
all_observations

array([[4.17684903e-01, 5.00000000e-01],
       [4.17163511e-01, 4.93296495e-01],
       [4.16124552e-01, 4.86642001e-01],
       [4.14575626e-01, 4.80085263e-01],
       [4.12528091e-01, 4.73674488e-01],
       [4.09996980e-01, 4.67457058e-01],
       [4.07184584e-01, 4.63840556e-01],
       [4.04111667e-01, 4.60491001e-01],
       [4.00800976e-01, 4.57433915e-01],
       [3.97277084e-01, 4.54692763e-01],
       [3.93566214e-01, 4.52288734e-01],
       [3.89696035e-01, 4.50240519e-01],
       [3.85695471e-01, 4.48564129e-01],
       [3.81594472e-01, 4.47272700e-01],
       [3.77423737e-01, 4.46376359e-01],
       [3.73214556e-01, 4.45882106e-01],
       [3.68998521e-01, 4.45793730e-01],
       [3.64807222e-01, 4.46111760e-01],
       [3.60672050e-01, 4.46833478e-01],
       [3.56623934e-01, 4.47952951e-01],
       [3.52693141e-01, 4.49461102e-01],
       [3.48908941e-01, 4.51345823e-01],
       [3.45299449e-01, 4.53592126e-01],
       [3.42618797e-01, 4.65534439e-01],
       [3.408867

In [178]:
all_observations_scaled

array([[0.49783672, 0.51990645],
       [0.38634907, 0.41872482],
       [0.32832671, 0.38859574],
       ...,
       [0.78312389, 0.96488556],
       [0.82143616, 0.89354939],
       [0.92623027, 0.89480476]])

In [179]:
vae(all_observations_scaled)

<tf.Tensor: shape=(10755, 2), dtype=float32, numpy=
array([[0.51328045, 0.51800674],
       [0.40487203, 0.42447254],
       [0.33217928, 0.3791114 ],
       ...,
       [0.7773116 , 0.9158826 ],
       [0.8025669 , 0.8813088 ],
       [0.85649186, 0.8805412 ]], dtype=float32)>

Random test to make sure posterior collapse is avoided

In [119]:
a = np.random.random((10, 2))
print(a)
vae.decoder(a)

[[0.12924743 0.30960184]
 [0.34146105 0.63325081]
 [0.2006226  0.23677983]
 [0.39715259 0.65883864]
 [0.33274673 0.92715291]
 [0.30385739 0.43389812]
 [0.20808629 0.38681099]
 [0.77297407 0.25049935]
 [0.63391301 0.06432549]
 [0.41720471 0.97503232]]


<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.2079564 , 0.3798874 ],
       [0.17498101, 0.44049573],
       [0.19405416, 0.3613656 ],
       [0.1657172 , 0.4435221 ],
       [0.18144417, 0.5038548 ],
       [0.17846933, 0.3992164 ],
       [0.19482864, 0.39318183],
       [0.10141916, 0.33977422],
       [0.11905786, 0.30723736],
       [0.16754702, 0.51014173]], dtype=float32)>

## MNIST

In [120]:
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train/255
x_train_flat = x_train.reshape(x_train.shape[0], x_train.shape[1]**2)

In [121]:
x_train.max()

1.0

In [122]:
enc = create_encoder(10, 784, [256, 128])
dec = create_decoder(10, 784, [256, 128])

vae = VAE(enc, dec, [0]*10, [1]*10, llik_scaling=100, kl_scaling=1)
vae.compile(optimizer=tf.keras.optimizers.Adam())

In [123]:
vae.fit(x_train_flat, batch_size=64, epochs=20)

Epoch 1/20


2022-08-07 13:54:56.950320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 10s 10ms/step - loss: 32.1690 - reconstruction_loss: 26.0431 - kl_loss: 1.0763
Epoch 2/20
938/938 [==============================] - 9s 10ms/step - loss: 24.7607 - reconstruction_loss: 22.7322 - kl_loss: 1.7090
Epoch 3/20
938/938 [==============================] - 9s 9ms/step - loss: 23.9416 - reconstruction_loss: 21.5231 - kl_loss: 2.2955
Epoch 4/20
938/938 [==============================] - 9s 9ms/step - loss: 23.3899 - reconstruction_loss: 20.2792 - kl_loss: 2.9727
Epoch 5/20
196/938 [=====>........................] - ETA: 7s - loss: 23.0395 - reconstruction_loss: 19.8954 - kl_loss: 3.1952

KeyboardInterrupt: 

In [ ]:
out = vae(x_train_flat)
out = out.numpy().reshape(out.shape[0], 28, 28)
out.shape

In [ ]:
example = 16
plt.imshow(out[example])

In [ ]:
plt.imshow(x_train[example])

In [ ]:
z = np.random.normal([0]*10, scale=[1]*10, size=(1, 10))
fake_num = vae.decoder(z).numpy().reshape(28,28)
plt.imshow(fake_num)